
##### 이 노트북은 CIFAR-10 데이터셋을 사용한 이미지 분류 실험입니다.

In [ ]:
# 새로운 셀 (맨 위)
%load_ext autoreload
%autoreload 2

print("✅ Auto-reload 활성화 - 파일 변경 자동 감지")

In [ ]:
# ============================================================
# 시드고정
# ============================================================
set_seed(config.SEED)

In [ ]:
# ============================================================
# 1. Python path 설정, 라이브러리 임포트
# ============================================================
import sys
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

%matplotlib inline

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root 생성 및 라이브러리 임포트 완료")

In [ ]:
# ============================================================
# 2. Config 설정
# ============================================================
from src.config import QuickTestConfig, DocumentConfig

# ============================================================
# Config 선택 (원하는 것만 주석 해제)
# ============================================================

# 방법 1: 빠른 테스트 (10% 데이터, 2 epoch, 2 fold)
config = QuickTestConfig()

# 방법 2: 문서 분류 대회 (전체 데이터, 100 epoch, 5 fold)
# config = DocumentConfig()

# ============================================================
# Config 출력
# ============================================================
config.print_config()

# ============================================================
# 앙상블 실험
# ============================================================

# 1단계: 빠른 실험 (단일 모델)
#config.update(N_FOLDS=1, USE_TTA=False)
# → 빠르게 augmentation, 하이퍼파라미터 테스트

# 2단계: 중간 검증 (기본 앙상블)
#config.update(N_FOLDS=5, USE_TTA=False)
# → 모델 성능 확인

# 3단계: 최종 제출 (풀 앙상블 + TTA)
#config.update(N_FOLDS=5, USE_TTA=True)
# → 최고 성능으로 제출
# ============================================================
# 세부 설정 수정 (필요시)
# ============================================================
# 일부 파라미터만 바꾸고 싶을 때 update() 사용

# 예시 1: Epoch와 Batch Size만 수정
# config.update(
#     EPOCHS=10,
#     BATCH_SIZE=128
# )

# 예시 2: 서브셋 비율 변경 (20%만 사용)
# config.update(
#     USE_SUBSET=True,
#     SUBSET_RATIO=0.2
# )

# 예시 3: 모델 변경
# config.update(
#     MODEL_NAME='resnet50',
#     IMAGE_SIZE=224
# )

# 예시 4: 학습률과 Early Stopping 조정
# config.update(
#     LR=0.0001,
#     PATIENCE=10
# )

# 예시 5: K-Fold 수 변경
# config.update(
#     N_FOLDS=5
# )

# 예시 6: Wandb 켜기
# config.update(
#     USE_WANDB=True
# )

# 예시 7: 여러 개 한번에 수정
# config.update(
#     MODEL_NAME='efficientnet_b0',
#     BATCH_SIZE=64,
#     EPOCHS=20,
#     LR=0.001,
#     N_FOLDS=3,
#     USE_SUBSET=True,
#     SUBSET_RATIO=0.15
# )

# ============================================================
# 또는 직접 속성 수정도 가능
# ============================================================
# config.EPOCHS = 10
# config.BATCH_SIZE = 64
# config.print_config()  # 변경사항 확인

In [ ]:
# ============================================================
# 3. 데이터 로드
# ============================================================
from src.data import load_data, get_train_augmentation, get_val_augmentation

# Config 기반으로 데이터 로드
train_dataset_raw, test_dataset, train_labels, class_names, num_classes = load_data(config)

# Config 업데이트 (클래스 수 자동 설정)
config.NUM_CLASSES = num_classes

device = config.DEVICE

print(f"\n✅ 데이터 로드 완료")
print(f"Device: {device}")
print(f"Train samples: {len(train_dataset_raw):,}")
print(f"Test samples: {len(test_dataset):,}")
print(f"Classes ({num_classes}): {class_names}")

In [ ]:
# ============================================================
# 4. 샘플 이미지 시각화 (랜덤)
# ============================================================
import random
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# 랜덤 인덱스 생성
random_indices = random.sample(range(len(train_dataset_raw)), 10)

for i, ax in enumerate(axes.flat):
    image, label = train_dataset_raw[random_indices[i]]
    ax.imshow(image)
    ax.set_title(f"{class_names[label]}")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 5. 모델 생성
# ============================================================
from src.model import get_model, print_model_info, print_model_list, get_optimizer
import torch.nn as nn

# 모델 리스트 보기 (선택 사항)
# print_model_list(config.MODEL_NAME)

# 모델 생성
model = get_model(
    model_name=config.MODEL_NAME,
    num_classes=config.NUM_CLASSES,  # config에서 가져오기
    pretrained=True
)
model = model.to(device)

# 모델 정보 출력
print_model_info(model, device, config.MODEL_NAME)

# Optimizer & Loss
optimizer = get_optimizer(model, config)
criterion = nn.CrossEntropyLoss()

print(f"\n✅ 학습 준비 완료")
print(f"Optimizer: Adam (lr={config.LR})")
print(f"Loss: CrossEntropyLoss")

In [ ]:
# ============================================================
# 6. Wandb 초기화
# ============================================================
wandb=None

if config.USE_WANDB:
    import wandb
    from src.model import MODEL_CONFIGS
    
    # Wandb 로그인 (최초 1회만, 주석 해제)
    # wandb.login()
    
    # 모델 display name
    model_display = MODEL_CONFIGS.get(
        config.MODEL_NAME, 
        {'display_name': config.MODEL_NAME}
    )['display_name']
    
    # 실험 이름
    run_name = f"{config.MODEL_NAME}_bs{config.BATCH_SIZE}_ep{config.EPOCHS}"
    if config.USE_SUBSET:
        run_name += f"_sub{int(config.SUBSET_RATIO*100)}"
    
    # Wandb 초기화
    wandb.init(
        project=config.WANDB_PROJECT,
        name=run_name,
        config=config.to_dict()  # Config 전체를 dict로 변환
    )
    
    print(f"✅ Wandb 초기화: {wandb.run.name}")
    print(f"📊 Project: {config.WANDB_PROJECT}")
else:
    print("⏭️  Wandb 비활성화")
    wandb = None  # 나중에 wandb 체크할 때 사용

In [ ]:
# ============================================================
# 7. K-Fold 학습
# ============================================================
from src.train import run_kfold_training

# Config 기반으로 K-Fold 학습 실행
fold_results = run_kfold_training(
    train_dataset_raw=train_dataset_raw,
    train_labels=train_labels,
    config=config  # config에 num_classes, device, use_wandb 모두 포함
)

print("\n" + "="*60)
print("🎉 K-Fold 학습 완료!")
print("="*60)

# Fold별 결과 요약
print("\n📊 Fold별 결과:")
for fold_idx, result in enumerate(fold_results, 1):
    print(f"  Fold {fold_idx}: Val Acc = {result['best_val_acc']:.4f}, "
          f"Val Loss = {result['best_val_loss']:.4f}")

# 평균 성능
avg_acc = sum(r['best_val_acc'] for r in fold_results) / len(fold_results)
avg_loss = sum(r['best_val_loss'] for r in fold_results) / len(fold_results)

print(f"\n🎯 평균 성능:")
print(f"  평균 Accuracy: {avg_acc:.4f}")
print(f"  평균 Loss: {avg_loss:.4f}")

In [ ]:
# ============================================================
# 8. 평가 및 결과 분석
# ============================================================
import platform
import matplotlib.pyplot as plt

# 한글 폰트 설정
if platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

from src.evaluation import run_full_evaluation

# Config 기반으로 전체 평가 실행
results = run_full_evaluation(
    fold_results=fold_results,
    test_dataset=test_dataset,
    class_names=class_names,
    config=config
)

# 결과 출력
print("\n" + "="*70)
print("🎉 최종 평가 결과")
print("="*70)
print(f"최종 테스트 정확도: {results['test_acc']:.2f}%")
print(f"최종 테스트 F1 Score: {results['test_f1']:.4f}")
print("="*70)

In [ ]:
# ============================================================
# 9. 제출파일 저장 (sample_submission.csv 기반)
# ============================================================
from src.submission import save_submission

# predictions를 포함한 results dict에서 예측값 추출
preds = results['predictions']

# sample_submission.csv 위치와 저장 경로 지정
sample_path = "sample_submission.csv"  # ← 실제 경로에 맞게 수정
save_path = "outputs/submission.csv"   # 결과 제출 파일 경로

save_submission(
    preds=preds,
    sample_path=sample_path,
    save_path=save_path
)


In [ ]:
# ============================================================
# 10. Wandb 종료 및 실험 완료
# ============================================================
if config.USE_WANDB:
    import wandb
    wandb.finish()
    print("\n✅ Wandb 종료 완료")
else:
    print("\n⏭️  Wandb 미사용")

print("\n" + "="*70)
print("🎉 모든 실험이 완료되었습니다!")
print("="*70)

In [ ]:
# ============================================================
# 10. 실험 결과 저장
# ============================================================
from src.logger import log_experiment_results

log_experiment_results(
    fold_results=fold_results,
    results=results,
    config=config
)

print("\n🎉 모든 작업 완료!")

In [ ]:
# ============================================================
# 11. 제출파일 저장 (sample_submission.csv 기반)
# ============================================================
from src.submission import save_submission

# predictions를 포함한 results dict에서 예측값 추출
preds = results['predictions']

# sample_submission.csv 위치와 저장 경로 지정
sample_path = "sample_submission.csv"  # ← 실제 경로에 맞게 수정
save_path = "outputs/submission.csv"   # 결과 제출 파일 경로

save_submission(
    preds=preds,
    sample_path=sample_path,
    save_path=save_path
)
